# 02_Process_Data
-------------
Lakeflow Declarative (formerly Delta Live Tables) Pipeline to pickup the data generated in notebook 01_SETUP, then clean, process, and transform it


### Bronze Layer


In [0]:
CREATE OR REPLACE MATERIALIZED VIEW customer_transactions
COMMENT "Raw transactions for customer purchases"
AS
SELECT
  *
FROM read_files(
  '/Volumes/users/landan_george/customer_segmentation/transactions/',
    format => "csv"
  );

### Silver Layer

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW customer_transactions_agg
COMMENT "Aggregated transactions for customer purchases"
AS
SELECT
  CustomerID,
  count(CustomerID) as TotalPurchases,
  ROUND(AVG(Amount), 2) as AvgOrderValue,
  ROUND(SUM(Amount), 2) as TotalSpent,
  datediff(current_date(), min(TransactionDate)) as DaysSinceFirstPurchase,
  datediff(current_date(), max(TransactionDate)) as DaysSinceLastPurchase,
  min(TransactionDate) as FirstPurchase,
  max(TransactionDate) as LastPurchase
FROM customer_transactions
GROUP BY CustomerID

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW customer_profile
COMMENT "Customer profile data for name, age, identity, etc."
AS
SELECT
  *
FROM read_files(
  '/Volumes/users/landan_george/customer_segmentation/customer_profile/',
    format => "csv"
  );

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW customer_engagement
COMMENT "Customer Engagement Data"
AS
SELECT
  *
FROM read_files(
  '/Volumes/users/landan_george/customer_segmentation/engagement/',
    format => "csv"
  );

### Gold Layer

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW customer_engagement_gold
COMMENT "Gold table for ML Model"
AS
SELECT
  t.CustomerID,
  t.TotalPurchases,
  t.AvgOrderValue,
  t.TotalSpent,
  t.DaysSinceFirstPurchase,
  t.DaysSinceLastPurchase,
  t.FirstPurchase,
  t.LastPurchase,
  p.FirstName,
  p.LastName,
  p.Age,
  p.Gender,
  p.City,
  CAST(split(p.EstLocation, ' ')[0] AS DOUBLE) as Lat,
  CAST(split(p.EstLocation, ' ')[1] AS DOUBLE) as Lon,
  p.SignupDate,
  e.EmailOpens,
  e.WebVisits,
  e.MobileAppSessions,
  e.LoyaltyScore,
  e.Returns,
  e.DiscountUsageRate
FROM customer_transactions_agg t
JOIN customer_profile p ON t.CustomerID = p.CustomerID
JOIN customer_engagement e ON t.CustomerID = e.CustomerID